# San Diego Police Department Stops per Race

In [119]:
%matplotlib inline
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib
import matplotlib.pyplot as plt
import seaborn as sns

### Introduction

With the death of George Floyd in police custody at the end of May 2020, the national conversation in the United States has once again shifted to the debate on police bias and systemic racism. As well as nearly every other major city, the city of San Diego has seen protests against its current police department, on the argument of unfair, racially biased police activity.  

Using data related to the demographics of San Diego's population and the stops made by the San Diego Police Department, this report will analyze data in an attempt to identify what, if any, biases may exist. This report is meant to determine what groups, if any, are disproportionately under or over represented in police stops of persons.

This report is independent and unfunded, without any affiliation to any groups or entities. This report is not intended to draw conclusions on why a group may or may not be over or under represented; the intention is to identify if any biases seem to exist from the available data. Any conclusions or observations not explicitly listed in this report are drawn at the discretion of the reader.

### Data

#### Data Sources

The data used for this report consists of San Diego Police Department (herein SDPD) data and US Census data. 

The SDPD data "is collected according to requirements set forth in Government Code section 12525.5 that was enacted as a result of the Racial and Identity Profiling Act of 2015 (AB 953), also known as RIPA." All of the data used is public and available at https://data.sandiego.gov/datasets/. 

Specifically, this report will use "Race of Persons Stopped" in order to establish the demographics of police stops (https://data.sandiego.gov/datasets/police-ripa-race/), and "Reason for Stop" to establish the reason for stopping a person (https://data.sandiego.gov/datasets/police-ripa-stop-reason/). 

US Census data will be used to define the racial demographic percentages of the city of San Diego, based on the 2010 US Census, as the 2020 Census is currently taking place. (https://web.archive.org/web/20120802190059/http://quickfacts.census.gov/qfd/states/06/0666000.html).

#### Data Limitations

The SDPD data on race is based on the stopping officer's perception of the stopped person's race, which may or may not be an accurate reflection with the race the stopped person identifies as / with. 

Both the data of SDPD and the Census do not have the same defined racial values, requiring certain values to be merged or omitted. Due to omission, only 97.6% of the Census data may be accurately used.

#### Data Sorting and Cleaning

##### Census Data

From the Census data, only the information relevant to the demographic makeup of San Diego will be used.

In order to have the same racial groups present in both Census and SDPD data, the Census values of "White" and "Multiracial" will be dropped, where "White / Non-Latino" will be renamed as "White". "Asian" and "Hawaiian / other Pacific Islander" will be merged into the group "Asian". The values for "Black / African American" will be renamed "Black", and the values for "Hispanic / Latino/a" will be renamed "Latinx", in respect to currently common self identification trends. "Native American" will remain unchanged.

In [50]:
# Demographics DF = df_dem
df_sdc = pd.read_excel('data/2010SDcensus.xls')
df_sdc = df_sdc[['People QuickFacts', 'San Diego']]
df_sdc = df_sdc.rename(columns = { 'People QuickFacts' : 'Race', 'San Diego' : 'Percent'})
df_dem = df_sdc.iloc[9:17]
df_dem['Percent'] = df_dem['Percent'].str.rstrip('%').astype('float')
df_dem = df_dem.reset_index(drop = True)
df_dem = df_dem.rename({0: 'White', 1: 'Black', 2: 'Native American', 3: 'Asian', 4: 'Hawaiian / other PI', 5: 'Multiracial', 6 : 'Latinx', 7 : 'White / non-Latinx'}, axis='index')
df_dem = df_dem.sort_values(by = 'Percent', ascending = False)
df_dem = df_dem.drop(['White', 'Multiracial', 'Asian', 'Hawaiian / other PI'])
new_row = {'Race': 'Asian, Native Hawaiian and Other Pacific Islander, percent, 2010(a)', 'Percent': 16.4}
df_dem = df_dem.append(new_row, ignore_index = True)
df_dem = df_dem.rename({2: 'Black',
                          3: 'Native American',
                          1: 'Latinx',
                          0: 'White',
                          4: 'Asian'},
                          axis='index')                    
df_dem = df_dem.sort_values(by = 'Percent', ascending = False)

/home/jupyterlab/conda/envs/python/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


##### SDPD Data

From the SDPD data, both data sets will be merged on their Stop ID, and any duplicates or values which are missing crucial information are dropped. The data set is made up of the shared "Stop ID", the "Race" of each person stopped from the "Race of Persons Stopped" data and the "Reason" of each stop from the "Reason for Stop" data.  

In order to match Census racial values, "Asian", "Middle Eastern or South Asian", and "Pacific Islander" will be merged into the group "Asian". Similarly, "Black" will be used for "Black/African American" and "Latinx" will be used for "Hispanic/Latino/a", for the same reasons mentioned with the Census Data. "White" and "Native American" will remain unchanged. The number of stops per race will be divided by the total number of stops, in order to establish a percent of stops that can be compared to the Census data.

The data values for "Reason for Stop" will be turned into percents, by racial makeup, of the total number of stops for a given reason. The two reasons that will be focused on are "Traffic Violation" and "Reasonable Suspicion", which together account for over 93% of all stops.

In [27]:
# Stop ID DF = df_sr
df_stop_reason = pd.read_csv('http://seshat.datasd.org/pd/ripa_stop_reason_datasd.csv')
df_race = pd.read_csv('http://seshat.datasd.org/pd/ripa_race_datasd.csv')
df_stop_reason = df_stop_reason[['stop_id','reason_for_stop']]
df_race = df_race[['stop_id','race']]
df_sr = pd.merge(df_race, df_stop_reason, on='stop_id')
df_sr.drop_duplicates(subset = 'stop_id', 
                     keep = False, inplace = True) 
df_sr = df_sr.reset_index(drop = True)
df_sr = df_sr.rename(columns = { 'stop_id' : 'Stop ID', 'race' : 'Race', 'reason_for_stop' : 'Reason'})
df_sr = df_sr.replace(['Asian', 'Middle Eastern or South Asian', 'Pacific Islander'], 'Asian')

In [4]:
# Percentage of stops per race = df_rc
per = df_sr['Race'].value_counts() / 253513 * 100
df_rc = pd.DataFrame(data=per)
df_rc = df_rc.reset_index(drop=False)
df_rc = df_rc.rename(columns = { 'index' : 'Race', 'Race' : 'Percent'})
df_rc['Percent'] = df_rc['Percent'].round(decimals=1)
df_rc = df_rc.rename({2: 'Black',
                          4: 'Native American',
                          1: 'Latinx',
                          0: 'White',
                          3: 'Asian'},
                          axis='index')  
df_rc = df_rc.sort_values(by = 'Percent', ascending = False)

In [ ]:
# DF for black race values = df_black
df_black = df_sr[df_sr['Race'] == 'Black/African American']
df_black = df_black.reset_index(drop = True)
df_black = df_black['Reason'].value_counts()
# DF for white race values = df_white
df_white = df_sr[df_sr['Race'] == 'White']
df_white = df_white.reset_index(drop = True)
df_white = df_white['Reason'].value_counts()
# DF for latinx race values = df_latinx
df_latinx = df_sr[df_sr['Race'] == 'Hispanic/Latino/a']
df_latinx = df_latinx.reset_index(drop = True)
df_latinx = df_latinx['Reason'].value_counts()
# DF for native american race values = df_na
df_na = df_sr[df_sr['Race'] == 'Native American']
df_na = df_na.reset_index(drop = True)
df_na = df_na['Reason'].value_counts()
# DF for asian race values = df_asian
df_asian = df_sr[df_sr['Race'] == 'Asian']
df_asian = df_asian.reset_index(drop = True)
df_asian = df_asian['Reason'].value_counts()
# df for reason of stop by race = df_reason
df_reason = pd.DataFrame(data = {'White' : df_white,
                     'Latinx' : df_latinx,
                     'Black' : df_black,
                     'Asian' : df_asian,
                     'Native American' : df_na})
df_reason = df_reason.reset_index(drop=False)
df_reason = df_reason.rename({0: 'Consented Search',
                          1: 'Violation of School Policy',
                          2: 'Truancy',
                          3: 'Warrant or Wanted Person',
                          4: 'Parole or Probation Related',
                          5: 'Education Code Discipline',
                          6: 'Reasonable Suspicion',
                          7: 'Traffic Violation'},
                          axis='index') 
df_reason = df_reason.rename(columns={'index' : 'Reason for Stop'})
df_reason['Total'] = df_reason.sum(axis=1)